## PyCity Schools Analysis

* Schools with smaller budgets per students (<\$585 per student) actually have shown better results than schools with higher budgets (\$645-675): 95% passing vs 73%.

* Students from smaller and medium sized schools (<2000 students) have better test results than students from large sized schools (2000-5000 students). Smaller and medium sized schools have Overall Passing Rate equal to 95% vs 76% in larger sized schools.


In [3]:
import pandas as pd
import numpy as np

In [4]:
schools = "Resources/schools_complete.csv"
students = "Resources/students_complete.csv"

schools_df = pd.read_csv(schools)
#schools_df

In [5]:
students_df = pd.read_csv(students)
#students_df.head()

In [7]:
total_schools = schools_df['School ID'].count()
total_students = students_df['Student ID'].count()
total_budget = schools_df['budget'].sum()
average_reading = students_df['reading_score'].mean()
average_math = students_df['math_score'].mean()
overall_passing_rate = (average_reading + average_math) / 2
math_over_70 = (students_df["math_score"] >= 70).sum()
passing_math = (math_over_70 * 100) / total_students
reading_over_70 = (students_df["reading_score"] >= 70).sum()
passing_reading = (reading_over_70 * 100) / total_students


## District Summary

In [8]:
totals = [{"Total Schools": total_schools,
           "Total Students": total_students, 
           "Total Budget": total_budget, 
           "Average Math Score":average_math, 
           "Average Reading Score":average_reading, 
           "% Passing Math": passing_math,
           "% Passing Reading":passing_reading, 
           "% Overall Passing Rate":overall_passing_rate}]
     
     
df_totals = pd.DataFrame(totals)
ordered_totals_df = df_totals[["Total Schools", "Total Students", "Total Budget", "Average Math Score",\
                               "Average Reading Score", "% Passing Math","% Passing Reading","% Overall Passing Rate"]]
ordered_totals_df["Total Budget"] = ordered_totals_df["Total Budget"].map("${:,.2f}".format)
ordered_totals_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,"$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.431606


## School Summary

In [55]:
grouped_students = students_df.groupby(['school_name'])
averages = grouped_students[['reading_score', 'math_score']].mean()

In [56]:
merged = pd.merge(averages, schools_df, how="left", on=["school_name", "school_name"])

In [57]:
reduced_merged = merged.drop(columns="School ID")

In [58]:
reduced_merged['Per Student Budget'] = (reduced_merged['budget']/reduced_merged['size'])#.map("${:,.2f}".format)

In [59]:

locked_reading_over_70 = students_df.loc[(students_df['reading_score'] >=70)]

grouped_locked_reading_over_70 =locked_reading_over_70.groupby(['school_name'])
#grouped_locked_reading_over_70.head()

final_grouped_licked_reading_over_70 = pd.DataFrame(grouped_locked_reading_over_70['reading_score'].count())
#final_grouped_licked_reading_over_70.head()

In [60]:
locked_math_over_70 = students_df.loc[(students_df['math_score'] >=70)]
grouped_locked_math_over_70 =locked_math_over_70.groupby(['school_name'])
final_grouped_licked_math_over_70 = pd.DataFrame(grouped_locked_math_over_70['math_score'].count())
#final_grouped_licked_math_over_70.head()

In [61]:
merged_temp = pd.merge(reduced_merged, final_grouped_licked_math_over_70, how="left", on=["school_name", "school_name"])
final_merged = pd.merge(merged_temp, final_grouped_licked_reading_over_70, how="left", on=["school_name", "school_name"])
#final_merged.head()

In [62]:
final_merged['% Passing Math']=(final_merged['math_score_y']*100) / final_merged['size']
final_merged['% Passing Reading']=(final_merged['reading_score_y']*100) / final_merged['size']
final_merged['% Overall Passing Rate']=(final_merged['% Passing Math'] + final_merged['% Passing Reading'])/2
#final_merged.head()

In [63]:
by_schools = final_merged.loc[:,['school_name','type','size','budget','Per Student Budget', 'math_score_x',\
                               'reading_score_x', '% Passing Math','% Passing Reading','% Overall Passing Rate']]


In [64]:
final_by_schools = by_schools.rename(columns={'school_name': 'School Name','type':'School Type', 'size':'Total Students',\
                                              'budget':'Total School Budget', 'math_score_x':'Average Math Score',\
                                              'reading_score_x':'Average Reading Score'})
final_by_schools['Per Student Budget']=final_by_schools['Per Student Budget'].map("${:,.2f}".format)

In [65]:
final_by_schools.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,Bailey High School,District,4976,3124928,$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
1,Cabrera High School,Charter,1858,1081356,$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
2,Figueroa High School,District,2949,1884411,$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
3,Ford High School,District,2739,1763916,$644.00,77.102592,80.746258,68.309602,79.299014,73.804308
4,Griffin High School,Charter,1468,917500,$625.00,83.351499,83.816757,93.392371,97.138965,95.265668


## Top Performing Schools (By Passing Rate)

In [20]:
top_schools = final_by_schools.sort_values(by='% Overall Passing Rate', ascending=False)
#top_schools.head()
new_index_top = top_schools.reset_index(drop=True)
new_index_top.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,Cabrera High School,Charter,1858,1081356,$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
1,Thomas High School,Charter,1635,1043130,$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
2,Pena High School,Charter,962,585858,$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
3,Griffin High School,Charter,1468,917500,$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
4,Wilson High School,Charter,2283,1319574,$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

In [21]:
bottom_schools = final_by_schools.sort_values(by='% Overall Passing Rate', ascending=True)
#top_schools.head()
new_index_botton = bottom_schools.reset_index(drop=True)
new_index_botton.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,Rodriguez High School,District,3999,2547363,$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
1,Figueroa High School,District,2949,1884411,$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
2,Huang High School,District,2917,1910635,$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
3,Johnson High School,District,4761,3094650,$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
4,Ford High School,District,2739,1763916,$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

In [134]:
nine = students_df.loc[students_df['grade'] == "9th"]

nine_by_schools = nine.loc[:,['school_name', 'grade','math_score']]
nine_by_schools_grouped = nine_by_schools.groupby(['school_name'])
nines = nine_by_schools_grouped.mean()
#nines.head()


In [135]:
ten = students_df.loc[students_df['grade'] == "10th"]
ten_by_schools = ten.loc[:,['school_name', 'grade','math_score']]
ten_by_schools_grouped = ten_by_schools.groupby(['school_name'])
tens = ten_by_schools_grouped.mean()

In [136]:
eleven = students_df.loc[students_df['grade'] == "11th"]
eleven_by_schools = eleven.loc[:,['school_name', 'grade','math_score']]
eleven_by_schools_grouped = eleven_by_schools.groupby(['school_name'])
elevens = eleven_by_schools_grouped.mean()

In [137]:
twelve = students_df.loc[students_df['grade'] == "12th"]
twelve_by_schools = twelve.loc[:,['school_name', 'grade','math_score']]
twelve_by_schools_grouped = twelve_by_schools.groupby(['school_name'])
twelves = twelve_by_schools_grouped.mean()

In [138]:
nine_ten = pd.merge(nines, tens, how="inner", on=["school_name"])
#nine_ten.head()

In [139]:
nine_ten_eleven = pd.merge(nine_ten, elevens, how="inner", on=["school_name"])
nine_ten_eleven = nine_ten_eleven.rename(columns={'math_score_x':'9th', 'math_score_y':'10th',\
                                     'math_score':'11th' })

In [140]:
grades = pd.merge(nine_ten_eleven, twelves, how="inner", on=["school_name"])
#grades.head()                                                                                   
                                                                                        

In [141]:
final_grades = grades.rename(columns={'math_score':'12th'})
final_grades.index.names=['School Name']

final_grades

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade

In [142]:
reading_nine_by_schools = nine.loc[:,['school_name', 'grade','reading_score']]
reading_nine_by_schools_grouped = reading_nine_by_schools.groupby(['school_name'])
reading_nines = reading_nine_by_schools_grouped.mean()

reading_ten_by_schools = ten.loc[:,['school_name', 'grade','reading_score']]
reading_ten_by_schools_grouped = reading_ten_by_schools.groupby(['school_name'])
reading_tens = reading_ten_by_schools_grouped.mean()

reading_eleven_by_schools = eleven.loc[:,['school_name', 'grade','reading_score']]
reading_eleven_by_schools_grouped = reading_eleven_by_schools.groupby(['school_name'])
reading_elevens = reading_eleven_by_schools_grouped.mean()

reading_twelve_by_schools = twelve.loc[:,['school_name', 'grade','reading_score']]
reading_twelve_by_schools_grouped = reading_twelve_by_schools.groupby(['school_name'])
reading_twelves = reading_twelve_by_schools_grouped.mean()
#reading_twelves.head()

In [143]:
reading_nine_ten = pd.merge(reading_nines, reading_tens, how="inner", on=["school_name"])
reading_nine_ten_eleven = pd.merge(reading_nine_ten, reading_elevens, how="inner", on=["school_name"])
reading_nine_ten_eleven = reading_nine_ten_eleven.rename(columns={'reading_score_x':'9th', 'reading_score_y':'10th',\
                                     'reading_score':'11th' })
reading_grades = pd.merge(reading_nine_ten_eleven, reading_twelves, how="inner", on=["school_name"])
#reading_grades.head()

In [144]:
reading_final_grades = reading_grades.rename(columns={'reading_score':'12th'})
reading_final_grades.index.names=['School Name']

reading_final_grades

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

In [225]:
bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [38]:
new_by_schools = by_schools.copy()
new_by_schools["Spending Ranges (Per Student)"] = pd.cut(by_schools["Per Student Budget"], bins, labels=group_names)
#new_by_schools.head()

In [244]:
by_budget = new_by_schools.loc[:,[ 'Spending Ranges (Per Student)', 'math_score_x','reading_score_x', '% Passing Math','% Passing Reading','% Overall Passing Rate']]
by_budget = by_budget.rename(columns={'math_score_x':'Average Math Score','reading_score_x':'Average Reading Score'})
#by_budget.head()

In [245]:
grouped_by_budget = by_budget.groupby('Spending Ranges (Per Student)').mean()
grouped_by_budget.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

In [43]:
bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [44]:
size_by_schools = by_schools.copy()
size_by_schools["School Size"] = pd.cut(by_schools["size"], bins, labels=group_names)
#size_by_schools.head()

In [45]:
by_size = size_by_schools.loc[:,[ 'School Size', 'math_score_x','reading_score_x', '% Passing Math','% Passing Reading','% Overall Passing Rate']]
by_size = by_size.rename(columns={'math_score_x':'Average Math Score','reading_score_x':'Average Reading Score'})
#by_size.head()

In [46]:
grouped_by_size = by_size.groupby('School Size').mean()
grouped_by_size.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

In [52]:
type_by_schools = by_schools.copy()
#type_by_schools.head()

In [53]:
by_type = type_by_schools.loc[:,[ 'type', 'math_score_x','reading_score_x', '% Passing Math','% Passing Reading','% Overall Passing Rate']]
by_type = by_type.rename(columns={'type':'School Type','math_score_x':'Average Math Score','reading_score_x':'Average Reading Score'})
#by_stype.head()

In [54]:
grouped_by_type = by_type.groupby('School Type').mean()
grouped_by_type.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
